# Wingbeat analysis
This notebook demonstrates how to analyse wingbeat data output from camfi.

First, load the required libraries.

In [ ]:
from bces import bces
import numpy as np



Next, load the wingbeat frequency data. It is in a tab-separated format with the following columns:

1. `image_name` : relative path to image
2. `capture_time` : datetime in yyyy-mm-dd HH:MM:SS format
3. `annotation_idx` : index of annotation in image (arbitrary)
4. `best_peak` : period of wingbeat in pixels
5. `blur_length` : length of motion blur in pixels
6. `snr` : signal to noise ratio of best peak
7. `wb_freq_up` : wingbeat frequency estimate, assuming upward motion (and zero body-length)
8. `wb_freq_down` : wingbeat frequency estimate, assuming downward motion (and zero body-length)
9. `et_up` : corrected moth exposure time, assuming upward motion
10. `et_dn` : corrected moth exposure time, assuming downward motion
11. `period_up` : wingbeat period, assuming upward motion (and zero body-length)
12. `period_dn` : wingbeat period, assuming downward motion (and zero body-length)
13. `spec_dens` : comma separated values, with the spectral density array associated with the annotation

For the purposes of this notebook, we are particularly interested in wingbeat frequency, so we will only load the relevant columns.

In [ ]:
data_path = "cabramurra_wingbeats.csv.bz2"

data = np.loadtxt(
    data_path,
    dtype=[
        ('image_name', 'U40'),
        ('capture_time', 'datetime64[s]'),
        ('annotation_idx', 'u1'),
        ('best_peak', 'u2'),
        ('blur_length', 'u2'),
        ('snr', 'f8'),
        ('wb_freq_up', 'f8'),
        ('wb_freq_dn', 'f8'),
        ('et_up', 'f8'),
        ('et_dn', 'f8')
    ],
    usecols=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9),
    delimiter='\t',
    skiprows=1,
)